In [ ]:
! pip install seaborn==0.11.0

In [ ]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style(style="darkgrid")
sns.set_palette(palette = 'pastel')
# color palette for seaborn
cmap = sns.diverging_palette(220, 10, as_cmap=True)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.simplefilter(action='ignore')

In [ ]:
sns.__version__

<center> At first, let's exploring mental health in the tech industry in 2014. Then, comparing results with data from 2016 and adding data from word-happiness in 2016. </center>

# <center>Exploring mental health in the tech industry in 2014</center>

Some information about explored data:

- "*This dataset is from a 2014 survey that measures attitudes towards mental health and frequency of mental health disorders in the tech workplace*". 
- **Features:**

<table>
<thead>
<tr><th>Feature name</th><th>Description</th></tr>
</thead>
<tbody>
<tr><td>Timestamp</td><td> - </td></tr>
<tr><td>Age</td><td> - </td></tr>
<tr><td>Gender</td><td> - </td></tr>
<tr><td>Country</td><td> - </td></tr>
<tr><td>State</td><td> (only for US) </td></tr>
<tr><td>Self employed</td><td> Are you self-employed? </td></tr>
<tr><td>Family history</td><td> Family history of mental illness </td></tr>
<tr><td>Treatment</td><td>Is treatment for a mental health condition was?</td></tr>
<tr><td>Work interfere</td><td> Is mental health condition affects work? </td></tr>
<tr><td>No employees</td><td> The number of employees in your company or organization </td></tr>
<tr><td>Remote work</td><td> Having remote work (outside of an office) at least 50% of the time </td></tr>
<tr><td>Tech company</td><td> The employer is primarily a tech company/organization </td></tr>
<tr><td>Benefits</td><td> Providing mental health benefits by the employer </td></tr>
<tr><td>Care options:</td><td> Providing options for mental health care by the employer </td></tr>
<tr><td>Wellness program</td><td> Discussion about mental health as part of an employee wellness program by the employes </td></tr>
<tr><td>Seek help</td><td> Providing resources by the employer to learn more about mental health issues and how to seek help </td></tr>
<tr><td>Anonymity</td><td> Protecting anonymity if you choose to take advantage of mental health or substance abuse treatment resources</td></tr>
<tr><td>Leave</td><td> How easy is it for you to take medical leave for a mental health condition? </td></tr>
<tr><td>Mental-health consequence: </td><td>  Having negative consequences caused by discussing a mental health issue with your employer</td></tr>
<tr><td>Phys-health consequence</td><td> Having negative consequences caused by discussing a physical health issue with your employer </td></tr>
<tr><td>Coworkers</td><td> Would you be willing to discuss a mental health issue with your coworkers?</td></tr>
<tr><td>Supervisor</td><td> Would you be willing to discuss a mental health issue with your direct supervisor(s)? </td></tr>
<tr><td>Mental health interview:</td><td> Would you bring up a mental health issue with a potential employer in an interview? </td></tr>
<tr><td>Phys health interview</td><td> Would you bring up a physical health issue with a potential employer in an interview? </td></tr>
<tr><td>Mental vs Physical</td><td> Do you feel that your employer takes mental health as seriously as physical health? </td></tr>
<tr><td>Obs consequence</td><td> Have you heard of or observed negative consequences for coworkers with mental health conditions in your workplace? </td></tr>
<tr><td>Comments</td><td> Any additional notes or comments </td></tr>
</tbody>
</table>






# **<center> Table of content: </center>**

**November, 2020.**

 - [Exploratory data analysis](#EDA:) 
 - [Feature Selection](##Feature-Selection:) 
 - ...
 
 
<p> <font size="3" color="red"> will be updated, if u find it useful, please upvote :) </font> </p>

# EDA:

Well, we have 27 columns (features) and 1259 rows (values - candidates, who answered the questions):

In [ ]:
data = pd.read_csv('/kaggle/input/mental-health-in-tech-survey/survey.csv')
data.head(2)

Have we got any *NULL-values*? Let's plot them:

In [ ]:
null_values = data.isnull().sum().sort_values(ascending=False).to_frame()
null_values = null_values.loc[null_values[0] != 0]
ax = sns.barplot(x=null_values.index, y=null_values[0], data=null_values,  palette="ch:.25")
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
ax = ax.set(ylabel="count", title='Parameters with NULL-values')

## Treatment, Family history and Work interfere

Explore our "target" value - treatment.

Candidates answered 'yes', if treatment is for a mental health condition was, and 'no' if treatment was not.


In [ ]:
treatment_values = data.treatment.value_counts().to_frame()
family_history_values = data.family_history.value_counts().to_frame()
plot_frame = pd.DataFrame({'answers': ['No', 'Yes'], 'treatment': treatment_values['treatment'], 'family_history': family_history_values['family_history']})
plot_frame = pd.melt(plot_frame, id_vars='answers', var_name="treatment", value_name="family_history")
sns.factorplot(x='treatment', y='family_history', hue='answers', data=plot_frame, kind='bar',  palette="ch:.25").set(ylabel='count', xlabel='', title='Answers count')

treatment_values_yes = data.treatment.loc[data.family_history == 'Yes'].value_counts().to_frame()
treatment_values_no = data.treatment.loc[data.family_history == 'No'].value_counts().to_frame()
plot_frame = pd.DataFrame({'family_history_yes': treatment_values_yes['treatment'], 'family_history_no': treatment_values_no['treatment']})
plot_frame.index.name = 'answers'
plot_frame.reset_index(inplace=True)
plot_frame = pd.melt(plot_frame, id_vars='answers', var_name="family_history_yes", value_name="family_history_no")
sns.factorplot(x='family_history_yes', y='family_history_no', hue='answers', data=plot_frame, kind='bar',  palette="ch:.25").set(ylabel='count', xlabel='', title='Treatment answers count')

work_interfere_yes = data.work_interfere.loc[data.treatment == 'Yes'].value_counts().to_frame()
work_interfere_no = data.work_interfere.loc[data.treatment == 'No'].value_counts().to_frame()
plot_frame = pd.DataFrame({'treatment_yes': work_interfere_yes['work_interfere'], 'treatment_no': work_interfere_no['work_interfere']})
plot_frame.index.name = 'answers'
plot_frame.reset_index(inplace=True)
plot_frame = pd.melt(plot_frame, id_vars='answers', var_name="treatment_yes", value_name="treatment_no")
sns.factorplot(x='treatment_yes', y='treatment_no', hue='answers', data=plot_frame, kind='bar',  palette="ch:.25").set(ylabel='count', xlabel='', title='Work interfere answers count')

work_interfere_yes = data.work_interfere.loc[data.family_history == 'Yes'].value_counts().to_frame()
work_interfere_no = data.work_interfere.loc[data.family_history == 'No'].value_counts().to_frame()
plot_frame = pd.DataFrame({'family_history_yes': work_interfere_yes['work_interfere'], 'family_history_no': work_interfere_no['work_interfere']})
plot_frame.index.name = 'answers'
plot_frame.reset_index(inplace=True)
plot_frame = pd.melt(plot_frame, id_vars='answers', var_name="family_history_yes", value_name="family_history_no")
sns.factorplot(x='family_history_yes', y='family_history_no', hue='answers', data=plot_frame, kind='bar',  palette="ch:.25").set(ylabel='count', xlabel='', title='Work interfere answers count')

## Age, Gender and Treatment

- At the first, we explore the "Gender" column;
- At the second, we explore the "Age" column;
- Exploring "treatment" from age-gender point.

Gender unique values:

In [ ]:
gender_values = data.Gender.value_counts().sort_values(ascending=False).to_frame()
gender_values = gender_values.rename(columns={'Gender': 'count'})
table_gender = gender_values.style.background_gradient(cmap=cmap)
table_gender

After cleaning "Gender"-column:

In [ ]:
# clean "Gender" column

data.Gender = data.Gender.str.lower()
male = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "cis male"]
female = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]
other = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", 
         "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", 
         "female (trans)", "queer", "ostensibly male, unsure what that really means", "p", "a little about you"]

data.Gender.loc[data.Gender.isin(male)] = 'male'
data.Gender.loc[data.Gender.isin(female)] = 'female'
data.Gender.loc[data.Gender.isin(other)] = 'others'

gender_values = data.Gender.value_counts().sort_values(ascending=False).to_frame()
gender_values = gender_values.rename(columns={'Gender': 'count'})
table_gender = gender_values.style.background_gradient(cmap=cmap)
table_gender

Checking age values:

Well, for the first, let's find outliers with  Percentile Capping (Winsorization) method:

    If a value exceeds the value of the 99th percentile and below the 1st percentile of given values are treated as outliers.

In [ ]:
from sklearn.ensemble import IsolationForest


def winsorization_outliers(df):
    out=[]
    for i in df:
        q1 = np.percentile(df , 1)
        q3 = np.percentile(df , 99)
        if i > q3 or i < q1:
            out.append(i)
    print("Outliers:",out)
    return out
    
outliers = winsorization_outliers(data.Age)

Dropping outliers and check distribution:

In [ ]:
# drop age-outliers

data_age = data.loc[~data.Age.isin(outliers)]
sns.histplot(data=data_age, x="Age")

"treatment" analysis  from the age-gender point:

In [ ]:
import plotly.express as px

fig = px.violin(data_age, y="Age", x="treatment", color="Gender", box=True, points="all")
fig.show()

- In both cases, the median man age is higher than female;

- If treatment was, the max men age is higher than women, and if treatment was not, the max women age is higher than men;

- Women and others have more cases in treatment-yes category.

Explore outliers data:

In [ ]:
data_outliers = data.loc[data.Age.isin(outliers)]

treatment_female = data_outliers.treatment.loc[data_outliers.Gender == 'female'].value_counts().to_frame()
treatment_male = data_outliers.treatment.loc[data_outliers.Gender == 'male'].value_counts().to_frame()

plot_frame = pd.DataFrame({'treatment_female': treatment_female['treatment'], 'treatment_male': treatment_male['treatment']})

plot_frame.index.name = 'answers'
plot_frame.reset_index(inplace=True)
plot_frame = pd.melt(plot_frame, id_vars='answers', var_name="treatment_female", value_name="treatment_male")
sns.factorplot(x='treatment_female', y='treatment_male', hue='answers', data=plot_frame, kind='bar',  palette="ch:.25").set(ylabel='count', xlabel='', title='Outliers treatment for male/female')

## Coutry, State

- Exploring country and state columns

In [ ]:
country_count = data.Country.value_counts().sort_values(ascending=False).to_frame()[:10]
country_count = country_count.rename(columns={'Country': 'count'})
plt.figure(figsize=(15,5))
ax = sns.barplot(x=country_count.index, y='count', data=country_count,  palette="ch:.25")
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
ax = ax.set_title('Top 10 countries')

In [ ]:
state_count = data.state.value_counts().sort_values(ascending=False).to_frame()[:10]
state_count = state_count.rename(columns={'state': 'count'})
plt.figure(figsize=(10,10))
ax = sns.barplot(x=state_count.index, y='count', data=state_count,  palette="ch:.25")
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
ax = ax.set_title('Top 10 States')

## Country, State and Treatment 

For most common countries:

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=data.Country.loc[data.treatment == 'Yes'].value_counts().index.to_list()[:10], values = data.Country.loc[data.treatment == 'Yes'].value_counts()[:10], name="Treatment -Yes"),
              1, 1)
fig.add_trace(go.Pie(labels=data.Country.loc[data.treatment == 'No'].value_counts().index.to_list()[:10], values = data.Country.loc[data.treatment == 'No'].value_counts()[:10], name="Treatment - No"),
              1, 2)

fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Country and trearment",
   
    annotations=[dict(text='Yes', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text='No', x=0.78, y=0.5, font_size=20, showarrow=False)])
fig.show()


fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=data.state.loc[data.treatment == 'Yes'].value_counts().index.to_list()[:10], values = data.state.loc[data.treatment == 'Yes'].value_counts()[:10], name="Treatment -Yes"),
              1, 1)
fig.add_trace(go.Pie(labels=data.state.loc[data.treatment == 'No'].value_counts().index.to_list()[:10], values = data.state.loc[data.treatment == 'No'].value_counts()[:10], name="Treatment - No"),
              1, 2)


fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="State and trearment",
   
    annotations=[dict(text='Yes', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text='No', x=0.78, y=0.5, font_size=20, showarrow=False)])
fig.show()

## Encoding features

All of the questions have categorical answers.

In the future, we could use categorical encoding for them. I use OrdinalEncoder for all columns with the exception of 'Timestamp', 'comments', 'Age'.

In [ ]:
from category_encoders.ordinal import OrdinalEncoder

data_encoding = data
encoder = OrdinalEncoder()
data_encoding = encoder.fit_transform(data.drop(['Timestamp', 'comments', 'Age'], axis=1))

In [ ]:
# new DataFrame
data_encoding['Age'] = data.Age
data_encoding.head()

# Feature Selection:

Trying different approaches  and comparing results:

- Correlation; 
- Chi-2;
- RFE;
- SMLR;
- ReliefF.

## Correlation

Correlation is a simple approach to find **linear** dependence between features.
- If correlation == |1| - two features have linear dependence;
- If correlation == 0 - two features have not linear dependence.

In [ ]:
corr = data_encoding.corr(method ='spearman')
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(15, 15))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

- High correlation is had between ['wellness_program' and 'seek_help'] and ['state' and 'country'];
- For our target value (treatment) high correlation have 'care_options', 'work interface', 'benefits'.

Correlation for 'treatment' column:

In [ ]:
f, ax = plt.subplots(figsize=(25, 1))
treatment = corr.sort_values(by=['treatment'], ascending=False).head(1).T
treatment = treatment.sort_values(by=['treatment'],ascending=False).T
sns.heatmap(treatment, cmap=cmap, annot=True)
plt.show()

In [ ]:
ten_corr_features = ['family_history', 'work_interfere', 'care_options', 'obs_consequence', 'benefits', 'mental_health_consequence', 
                     'Country', 'anonymity', 'mental_health_interview', 'leave']

In [ ]:
print('Top 10 features: \n', ten_corr_features)

## Chi-2

Checks if there is a significant difference between the observed and expected frequencies of two categorical numbers. Thus, the null hypothesis that there is no relationship between the variables is tested:

<center> $X^{2}=\frac{(\textrm{Observed frequency} - \textrm{Expected frequency})^2}{\textrm{Expected frequency}}$ </center>

I want to use top-10 features:

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

features_norm = MinMaxScaler().fit_transform(data_encoding.drop(['treatment'], axis=1))
chi_selector = SelectKBest(chi2, k=10)
chi_selector.fit(features_norm, data_encoding.treatment)

chi_support = chi_selector.get_support()
chi_feature = data_encoding.drop(['treatment'], axis=1).loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

In [ ]:
print('Top-10 features: \n', chi_feature)

## RFE

**RFE (Recursive Feature Removal)**: 

A greedy search algorithm that selects features by recursively defining ever smaller feature sets. It ranks the features according to the order in which they are removed.

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=10, step=10, verbose=5)
rfe_selector.fit(features_norm ,data_encoding.treatment)

In [ ]:
rfe_support = rfe_selector.get_support()
rfe_features = data_encoding.drop(['treatment'], axis=1).loc[:,rfe_support].columns.tolist()
print('Top-10 features: \n', rfe_features)

## SMLR

**SLMR(Sparse Logistic Regression MULTINOMIAL, sparse multinomial logistic regression):**

This algorithm implements the l1-regularization using ARD (Automatic relevance determination, automatic determination of relevance ) in the classical multinomial logistic regression. Regularization determines the importance of each feature and nullifies those that are useless for forecasting.

In [ ]:
! pip install git+https://github.com/KamitaniLab/smlr.git

In [ ]:
import smlr
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_encoding.drop(['treatment'], axis=1), data_encoding.treatment, test_size=0.33, random_state=42)

model = smlr.SMLR(max_iter=1000, tol=1e-8, verbose=5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

In [ ]:
features_weight = model.coef_[0]
features_index = [id for id, weight in enumerate(features_weight) if weight!=0]
smlr_features = data_encoding.drop(['treatment'], axis=1).iloc[:, features_index].columns.to_list()
print('Top SLMR features: \n', smlr_features)

## Relief

This method samples randomly instances from the dataset and updates the relevance of each feature based on the difference between the selected instance and the two nearest instances of the same and opposite classes. If a feature difference is observed in the neighboring instances of the same class ( a ‘hit’), the feature score decreases, alternatively if the feature value difference is observed with a different score (a ‘miss’) then the feature score increases.

![](https://miro.medium.com/max/472/1*EjkXuRcsUqQyQ-sw8uCM8w.png)

The extended algorithm, ReliefF applies feature weighting and searches for more nearest neighbors.

In [ ]:
! pip install ReliefF

In [ ]:
from ReliefF import ReliefF

fs = ReliefF(n_neighbors=1, n_features_to_keep=10)
features = fs.fit_transform(data_encoding.drop(['treatment'], axis=1).to_numpy(), data_encoding['treatment'].to_numpy()).T

In [ ]:
columns = data_encoding.drop(['treatment'], axis=1).columns.to_list()
reliefF_features = []
for feature in features:
    for column in columns:
        idx = (data_encoding.drop(['treatment'], axis=1)[column] == feature)
        if idx.all() == True:
            reliefF_features.append(column)

print('Top-10 features: \n', reliefF_features)

## Comparing:

In [ ]:
feature_names = pd.DataFrame({'features': columns})
feature_names = feature_names.set_index('features')
feature_names[['correlation', 'chi_2', 'rfe', 'smlr', 'reliefF']] = np.nan

In [ ]:
def is_important_feature(row, features):
    names = []
    for index in row.index.to_list():
        if index in features:
            names.append(index)
        else:
            names.append(np.NaN)
    return names

In [ ]:
feature_names.correlation = feature_names.apply(lambda row: is_important_feature(row, ten_corr_features))
feature_names.chi_2 = feature_names.apply(lambda row: is_important_feature(row, chi_feature))
feature_names.rfe = feature_names.apply(lambda row: is_important_feature(row, rfe_features))
feature_names.smlr = feature_names.apply(lambda row: is_important_feature(row, smlr_features))
feature_names.reliefF = feature_names.apply(lambda row: is_important_feature(row, reliefF_features))

In [ ]:
feature_names

In [ ]:
feature_names['counts'] = 5 - feature_names.isnull().sum(axis=1)

In [ ]:
plt.figure(figsize=(20,5))
features_counts = feature_names['counts'].sort_values(ascending=False).to_frame()
ax = sns.barplot(x=features_counts.index, y='counts', data=features_counts,  palette="ch:.25")
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.1f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
ax = ax.set_xticklabels(features_counts.index, rotation = 45, ha="right")


On the bar plot we can see the most important features.
They are 
- obs_consequence, wich means having heard of or observed negative consequences for coworkers with mental health conditions in workplace;
- family history, which means family history of mental illness;
- work_interfere: "Is mental health condition affects work?;
- care options, which means providing options for mental health care by the employer;
- anonymity, which means protecting anonymity if you choose to take advantage of mental health or substance abuse treatment resources;
- country;
- gender;
- benefits, which means providing mental health benefits by the employer;
- mental_health_consequence, which means having negative consequences caused by discussing a mental health issue with your employer.

Explore important features with count = 5:

In [ ]:
import plotly.express as px


fig = px.parallel_categories(data[['treatment', 'obs_consequence', 'family_history',  
                                   'work_interfere']])
fig.show()

Explore important features with count = 4:

In [ ]:
fig = px.parallel_categories(data[['treatment', 'care_options', 
                                   'anonymity', 'Gender', 'benefits', 'mental_health_consequence']])
fig.show()